In [ ]:
import numpy as np
import pandas as pd
import gc

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from xgboost import XGBClassifier
from xgboost import plot_importance

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1>Tabular Playground Series - May 2022 - Modelling</h1>


In [ ]:
def evaluate_model(model, x, y):
    y_pred_prob = model.predict_proba(x)[:, 1]
    auc_roc = roc_auc_score(y, y_pred_prob)
    return {'auc_roc_curve' : auc_roc}

In [ ]:
def encode_sequence_of_chars(df, feature):
    return df[feature].map(lambda string: np.std(list(map(ord, string))))

In [ ]:
def get_xgb_baseline(params={}):
    return XGBClassifier(**params,
                         objective='binary:logistic',
                         tree_method='gpu_hist',
                         predictor='gpu_predictor',
                         random_state=seed,
                         verbosity=0)

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/train.csv')

In [ ]:
train_df.drop('id', axis=1, inplace=True)
target = train_df.pop('target')

In [ ]:
seed = 47

<h2>1. Feature engineering<h2>

<h3>1.1 Encoding the feature f_27</h3>

In [ ]:
train_df['f_27'] = encode_sequence_of_chars(train_df, 'f_27')

<h3>1.2 Adding Synthetic features</h3>

In [ ]:
geomean = lambda x, axis : np.exp(np.mean(np.log(x), axis=axis))
harmonic_mean = lambda x, axis : len(x) / np.sum(1.0/x, axis=axis) 

funcs = {'mean' : np.mean, 
         'std' : np.std, 
         'var' : np.var, 
         'geo_mean' : geomean, 
         'harmonic_mean' : harmonic_mean, 
         'median' : np.median,
         'none': None}

results, names = list(), list()

for key in funcs.keys():
    x_train = train_df.copy()
    y_train = target
    if funcs[key] is not None:
        x_train[key] = funcs[key](x_train, axis=1)
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2, random_state=seed, shuffle=False)       
    model = get_xgb_baseline()

    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])
    
for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

index_best = np.argmax(results)
print('Best result is of:', names[index_best], 'with score:', results[index_best])


<h2>2. Spliting the dataset into train/test with sizes 80%-20%</h2> 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=seed, shuffle=False)

<h2>3. Tabular Playground Series - Apr 2022 - XGBoost</h2>

<h3>3.1 Baseline - XGBoost</h3>

In [ ]:
model = XGBClassifier(objective='binary:logistic',
                      tree_method='gpu_hist',
                      predictor='gpu_predictor',
                      random_state=seed,
                      verbosity=0)

model.fit(x_train, y_train)
score = evaluate_model(model, x_test, y_test)
print(score)

<h3>3.1 Manual Grid Search - XGBoost</h3>

In [ ]:
train_df['mean'] = np.mean(train_df, axis=1)
x_train, x_test, y_train, y_test = train_test_split(train_df, target, test_size=0.2, random_state=seed, shuffle=False)
params = {}

<h4>3.1.1 - Testing different number of estimators<h4>

In [ ]:
results_trees = {}
trees = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 950, 975, 1000, 1025, 1050, 1100, 1150, 1290, 1295, 1300, 1305, 1310, 1315, 1325, 2000, 3000, 4000, 5000, 10000]

for n in trees:
    params['n_estimators'] = n
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_trees[n] = result['auc_roc_curve']
    print('n_estimators:', n, 'auc_roc_score:', results_trees[n])

best_nestimator = max(results_trees, key=results_trees.get)
print('\nBest n_estimators:', best_nestimator, 'AUC ROC score:', results_trees[best_nestimator])

<h4>3.1.2 - Testing different max_depth</h4>

In [ ]:
results_max_depths = {}
params['n_estimators'] = best_nestimator
max_depths = [i for i in range(1,25)]

for max_depth in max_depths:
    params['max_depth'] = max_depth
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_max_depths[max_depth] = result['auc_roc_curve']
    print('max_depth:', max_depth, 'auc_roc_score:', results_max_depths[max_depth])

best_max_depth = max(results_max_depths, key=results_max_depths.get)
print('\nBest max_depth:', best_max_depth, 'AUC ROC score:', results_max_depths[best_max_depth])

<h4>3.1.3 - Testing different subsamples</h4>


In [ ]:
results_subsamples = {}
params['max_depth'] = best_max_depth
subsamples = [i for i in np.arange(0.1, 1.1, 0.1)]

for subsample in subsamples:
    params['subsample'] = subsample
    model = model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_subsamples[subsample] = result['auc_roc_curve']
    print('subsample:', subsample, 'auc_roc_score:', results_subsamples[subsample])

best_subsample = max(results_subsamples, key=results_subsamples.get)
print('\nBest subsample:', best_subsample, 'AUC ROC score:', results_subsamples[best_subsample])

<h4>3.1.4 - Testing different learning rates</h4>

In [ ]:
results_etas = {}
params['subsample'] = best_subsample
etas = [0.0001, 0.001, 0.003, 0.005, 0.01, 0.03, 0.05, 0.1, 0.12, 0.13, 0.3, 0.5, 1.0]

for eta in etas:
    params['eta'] = eta
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_etas[eta] = result['auc_roc_curve']
    print('eta:', eta, 'auc_roc_score:', results_etas[eta])

best_eta = max(results_etas, key=results_etas.get)
print('\nBest eta:', best_eta, 'AUC ROC score:', results_etas[best_eta])

<h4>3.1.5 - Testing different number of features<h4>

In [ ]:
results_colsample_bytrees = {}
params['eta'] = best_eta
colsample_bytrees = [i for i in np.arange(0.1, 1.1, 0.1)]

for colsample_bytree in colsample_bytrees:
    params['colsample_bytree'] = colsample_bytree
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_colsample_bytrees[colsample_bytree] = result['auc_roc_curve']
    print('colsample_bytree:', colsample_bytree, 'auc_roc_score:', results_colsample_bytrees[colsample_bytree])

best_colsample_bytree = max(results_colsample_bytrees, key=results_colsample_bytrees.get)
print('\nBest colsample_bytree:', best_colsample_bytree, 'AUC ROC score:', results_colsample_bytrees[best_colsample_bytree])

<h4>3.1.6 - Testing different values for min_child_weight<h4>

In [ ]:
results_min_child_weight = {}
params['colsample_bytree'] = best_colsample_bytree
min_child_weights = [i for i in range(1,10)]

for min_child_weight in min_child_weights:
    params['min_child_weight'] = min_child_weight
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_min_child_weight[min_child_weight] = result['auc_roc_curve']
    print('min_child_weight:', min_child_weight, 'auc_roc_score:', results_min_child_weight[min_child_weight])

best_min_child_weight = max(results_min_child_weight, key=results_min_child_weight.get)
print('\nBest min_child_weight:', best_min_child_weight, 'AUC ROC score:', results_min_child_weight[best_min_child_weight])

<h4>3.1.7 - Testing different values for gamma<h4>

In [ ]:
results_gamma = {}
params['min_child_weight'] = best_min_child_weight
gammas = [0, 0.01, 0.02, 0.03, 0.1, 0.3, 0.5, 1, 1.1, 1.5, 2, 5, 7, 9, 10]

for gamma in gammas:
    params['gamma'] = gamma
    model = get_xgb_baseline(params)
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    results_gamma[gamma] = result['auc_roc_curve']
    print('gamma:', gamma, 'auc_roc_score:', results_gamma[gamma])

best_gamma = max(results_gamma, key=results_gamma.get)
print('\nBest gamma:', best_gamma, 'AUC ROC:', results_gamma[best_gamma])

In [ ]:
params['gamma'] = best_gamma
print('Best Hyperparameters:', params)

<h2>4. Submission</h2>

<h2>4.1 Train model with the best found params</h2>


In [ ]:
model = get_xgb_baseline(params)
model.fit(x_train, y_train)
result = evaluate_model(model, x_test, y_test)
print(result)

<h3>Importance features plot</h3>


In [ ]:
plot_importance(model)

In [ ]:
del train_df, x_train, y_train, x_test, y_test
gc.collect()

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')

In [ ]:
ids = test_df.pop('id')
test_df['f_27'] = encode_sequence_of_chars(test_df, 'f_27')
test_df['mean'] = np.mean(test_df, axis=1)

In [ ]:
target = model.predict_proba(test_df)[:,1]
submission = pd.DataFrame({'id' : ids, 'target' : target})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)